# Gluing and Merging

Within this notebook, we glue the individual files which each contain a single sweep (elevation level), into full volume scans, where each file represents a volume scan. 

We have downloaded the raw data from the [ARM Data portal](https://adc.arm.gov/discovery/#/results/datastream::gucxprecipradarS2.00), using the `gucxprecipradarS2.00` datastream.

In [16]:
import os
import time
import datetime
import numpy as np
import glob
from dask.distributed import Client, LocalCluster, progress

import pyart

## 1) Define Functions

In [2]:
def radar_glue(b_radar, radar_list):
    for rad in radar_list:
        b_radar = pyart.util.join_radar(b_radar, rad)
    
    return b_radar

In [3]:
def volume_from_list(vlist):
    try:
        base_radar = pyart.io.read(vlist[0])
        radars = [pyart.io.read(sw) for sw in vlist[1::]]
        glue = radar_glue(base_radar, radars)
        del base_radar
        del radars
    except:
        glue = []
        pass
    
    return glue

In [4]:
def granule(Dvolume):
    #OUTPUT_DIR = '/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202203_glue/'
    OUTPUT_DIR = '/lus/eagle/projects/CPOL/obrienj/202209_glued/'
    if len(Dvolume) == 8:
        try:
            base_rad = pyart.io.read(Dvolume[0])

            out_radar = volume_from_list(Dvolume)
            if out_radar:
                ff = time.strptime(out_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
                dt = datetime.datetime.fromtimestamp(time.mktime(ff)) + datetime.timedelta(seconds = int(out_radar.time['data'][0]))
                strform = dt.strftime(OUTPUT_DIR + 'xprecipradar_guc_volume_%Y%m%d-%H%M%S.b1.nc')
                print(strform)
                #FIX for join issue.. to be fixed in Py-ART
                out_radar.sweep_mode['data'] = np.tile(base_rad.sweep_mode['data'], N_TILTS)
                nwrite = pyart.io.write_cfradial(strform, out_radar)
                del out_radar
                del nwrite
            del base_rad
        except:
            print("FAILED GRANULE")
            pass

In [5]:
def test(Dvolume):
    print(Dvolume[0], len(Dvolume[0]), DATA_DIR)

## 2) Define Processing Variables

In [6]:
# Define location of the raw data - NOTE: Must be untarred!
DATA_DIR = '/lus/eagle/projects/CPOL/obrienj/202209_nc/'
#DATA_DIR = '/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/nc_files/202203_nc/'
# Define the location to output the data to
OUTPUT_DIR = '/lus/eagle/projects/CPOL/obrienj/202209_glued/'
#OUTPUT_DIR = '/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202203_glue/'
# Define the suffix of the base scan
BASE_SCAN_PPI = '1_PPI.nc'
# Define the desired suffix of the volume file
PPI_PATTERN = 'PPI.nc'
# Define the number of elevation levels
N_TILTS = 8

In [7]:
# Select the days to process
DAY = 'gucxprecipradarS2.00.' + '202209*'

## 3) Create Volume Scans

In [8]:
# sort the input files
all_files = glob.glob(DATA_DIR + DAY)
all_files.sort()

In [9]:
# Iterate over the files within the directory.
# Determine which are base scans and which are ppi scans
# NOTE: There are RHI scans within the tar file not used.
base_scans = []
volumes = []
ppis = []
in_volume = False
for file in all_files:
    if PPI_PATTERN in file:
        ppis.append(file)
    if BASE_SCAN_PPI in file:
        base_scans.append(file)

In [10]:
# Determine the scan volumes
volumes = []
for base in base_scans:
    base_scan_index = np.where(np.array(ppis) == base)[0][0]
    volume = ppis[base_scan_index: base_scan_index + N_TILTS]
    volumes.append(volume)

In [11]:
granule(volumes[0])

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-000533.b1.nc


In [12]:
# Start up a Dask Cluster for Processing the Granule function
# NOTE: 96 GB is all that is available on OLCF Open Jupyter
#     requires openning 8 radar objects at once; ~4.5 GB per worker
#     Need at least 10 GB memory limit I think...
cluster = LocalCluster(n_workers=32, threads_per_worker=1, processes=1)

#cluster.scale(8)  # Sets the number of workers to 10
#cluster.adapt(minimum=4, maximum=8)
client = Client(cluster)
client

/home/jrobrien/.conda/envs/jhub_act_pyart/lib/python3.10/site-packages/tornado/platform/asyncio.py:193: DeprecationWarning: There is no current event loop
  old_loop = asyncio.get_event_loop()
/home/jrobrien/.conda/envs/jhub_act_pyart/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41272 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41272/status,
Dashboard: http://127.0.0.1:41272/status,Workers: 32
Total threads: 32,Total memory: 503.60 GiB
Status: running,Using processes: 1
Comm: tcp://127.0.0.1:39041,Workers: 32
Dashboard: http://127.0.0.1:41272/status,Total threads: 32
Started: Just now,Total memory: 503.60 GiB
Comm: tcp://127.0.0.1:35256,Total threads: 1
Dashboard: http://127.0.0.1:43142/status,Memory: 15.74 GiB
Nanny: tcp://127.0.0.1:38879,


In [13]:
# Use Dask distributed map utility to call the granule function
future = client.map(granule, volumes)

In [15]:
%time
data = client.gather(future)

CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 26.5 µs


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: Deprecation

ERROR 1: PROJ: proj_create_from_database: Open of /home/jrobrien/.conda/envs/jhub_act_pyart/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /home/jrobrien/.conda/envs/jhub_act_pyart/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /home/jrobrien/.conda/envs/jhub_act_pyart/share/proj failed



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-000533.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-001133.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-001733.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-002333.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-002932.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220901-003533.b1.nc
/l


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-205348.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-205947.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-210548.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-211148.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-211748.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220902-212348.b1.nc
/l

...

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-181843.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-182443.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-183043.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-183643.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-184243.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-184843.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-185443.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-190043.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-190643.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-191243.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220909-1

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-164250.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-164850.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-165450.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-170050.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-170650.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-171250.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-171850.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-172450.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-173050.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-173650.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220910-1

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-120906.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-121506.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-122106.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-122706.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-123306.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-123906.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-124506.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-125106.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-125706.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-130306.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220927-1

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-084328.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-084928.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-085528.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-090128.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-090728.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-091328.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-091928.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-092528.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-093128.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-093728.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220915-0

/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-133105.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-133705.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-134305.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-134905.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-135505.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-140105.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-140705.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-141305.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-141906.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-142506.b1.nc
/lus/eagle/projects/CPOL/obrienj/202209_glued/xprecipradar_guc_volume_20220912-1